In [9]:
# ===============================================
# 1) IMPORTS ET PARAMÈTRES
# ===============================================
import pandas as pd
import numpy as np
from pathlib import Path

# Chemins
data_dir = Path("../data")
output_dir = Path("../csv_clean")
output_dir.mkdir(exist_ok=True)

# Fichiers valeurs foncières
valeurs_files = {
    2022: data_dir / "valeurs_foncieres_2022.csv",
    2023: data_dir / "valeurs_foncieres_2023.csv",
    2024: data_dir / "valeurs_foncieres_2024.csv",
}

# Espaces verts / écoles / transports
parcs_path = data_dir / "parcs_jardins_communes_clean.csv"
colleges_path = data_dir / "colleges.csv"
lycees_path = data_dir / "lycees.csv"
transports_path = data_dir / "transports.csv"

# ===============================================
# 2) CHARGEMENT ET NETTOYAGE DES DONNÉES FONCIÈRES
# ===============================================
def charger_valeurs_foncieres(fichier, annee):
    df = pd.read_csv(fichier, low_memory=False)
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    
    cols_utiles = [
        "date_mutation", "valeur_fonciere", "code_commune",
        "nom_commune", "surface_reelle_bati", "nombre_pieces_principales", "type_local"
    ]
    df = df[[c for c in cols_utiles if c in df.columns]].copy()
    
    df = df.dropna(subset=["valeur_fonciere", "surface_reelle_bati", "code_commune"])
    df["valeur_fonciere"] = pd.to_numeric(df["valeur_fonciere"], errors="coerce")
    df["surface_reelle_bati"] = pd.to_numeric(df["surface_reelle_bati"], errors="coerce")
    df["prix_m2"] = df["valeur_fonciere"] / df["surface_reelle_bati"]
    df["annee"] = annee
    df["code_commune"] = df["code_commune"].astype(str).str.zfill(5)
    df = df[df["type_local"].isin(["Maison", "Appartement"])]
    df = df[df["prix_m2"].between(500, 15000)]
    return df

# Fusion des années
dfs = [charger_valeurs_foncieres(path, annee) for annee, path in valeurs_files.items()]
df_all = pd.concat(dfs, ignore_index=True)
print(f"✅ Données chargées : {len(df_all):,} lignes")

# ===============================================
# 3) STATS PAR COMMUNE (avant filtrage)
# ===============================================
prix_m2_commune = (
    df_all.groupby(["annee","code_commune","nom_commune"], as_index=False)
          .agg(prix_m2_median=("prix_m2","median"),
               prix_m2_moyen =("prix_m2","mean"),
               nb_ventes     =("prix_m2","count"))
)

pivot = prix_m2_commune.pivot(index="code_commune", columns="annee", values="prix_m2_median")
# évolution % seulement si on a 2022 et 2024
pivot["evolution_2022_2024_%"] = ((pivot.get(2024) - pivot.get(2022)) / pivot.get(2022) * 100)
pivot = pivot.reset_index()

# médiane & moyenne communales (toutes années confondues)
stats_commune = (
    df_all.groupby(["code_commune"], as_index=False)
          .agg(prix_m2_median_commune=("prix_m2","median"),
               prix_m2_moyen_commune =("prix_m2","mean"))
)

# ===============================================
# 4) ENRICHISSEMENT (fusion des indicateurs)
# ===============================================
df_merge = (
    df_all
    .merge(pivot[["code_commune","evolution_2022_2024_%"]], on="code_commune", how="left")
    .merge(stats_commune, on="code_commune", how="left")
)

# Espaces verts (facultatif)
try:
    parcs = pd.read_csv(parcs_path)
    parcs["code_commune"] = parcs["code_commune"].astype(str).str.zfill(5)
    df_merge = df_merge.merge(parcs[["code_commune","score_espaces_verts"]], on="code_commune", how="left")
except Exception:
    pass

# ===============================================
# 5) FILTRAGE PERSONA + DIAGNOSTIC
# ===============================================
def show(n, label): print(f"{n:>8}  {label}")

df_f = df_merge.copy()
show(len(df_f), "lignes au départ")

# 1) ≥ 4 pièces
df_f = df_f[df_f["nombre_pieces_principales"] >= 4]
show(len(df_f), "après ≥ 4 pièces")

# 2) Bon rapport Q/P : ≤ médiane * 1.15
df_f = df_f[df_f["prix_m2"] <= df_f["prix_m2_median_commune"] * 1.15]
show(len(df_f), "après prix <= médiane*1.15")

# 3) Peu de travaux : ≥ 0.5 * moyenne
df_f = df_f[df_f["prix_m2"] >= 0.5 * df_f["prix_m2_moyen_commune"]]
show(len(df_f), "après prix >= 0.5*moyenne")

# 4) Potentiel LT : garder NaN (pas d’info) + communes pas trop baissières
#    -> on autorise NaN OU évolution >= quantile 25% OU > -25%
evo = df_f["evolution_2022_2024_%"]
seuil_evo = evo.quantile(0.25) if evo.notna().sum() else -25
df_f = df_f[(evo.isna()) | (evo >= seuil_evo) | (evo > -25)]
show(len(df_f), f"après filtre évolution (seuil={seuil_evo:.1f}% ou NaN)")

# 5) Espaces verts : au-dessus du 30e percentile si dispo
if "score_espaces_verts" in df_f.columns:
    sev = df_f["score_espaces_verts"].quantile(0.30)
    df_f = df_f[df_f["score_espaces_verts"].fillna(sev) >= sev]
    show(len(df_f), "après filtre espaces verts (>= P30)")

# 6) Nettoyage final
df_f = df_f.drop_duplicates(subset=["date_mutation","valeur_fonciere","code_commune"]).copy()
cols_finales = [
    "date_mutation","valeur_fonciere","code_commune","nom_commune",
    "surface_reelle_bati","nombre_pieces_principales","type_local","prix_m2",
    "prix_m2_median_commune","prix_m2_moyen_commune","evolution_2022_2024_%",
]
if "score_espaces_verts" in df_f.columns:
    cols_finales.append("score_espaces_verts")
df_f = df_f[cols_finales]

show(len(df_f), "TOTAL retenu (persona)")


# ===============================================
# 6) EXPORT DU CSV FINAL
# ===============================================
df_f.to_csv(output_dir / "valeurs_foncieres_persona.csv", index=False)
print(f"✅ Exporté : csv_clean/valeurs_foncieres_persona.csv ({len(df_f)} lignes)")

# Aperçu final
display(df_f.head(10))



✅ Données chargées : 79,497 lignes
   79497  lignes au départ
   34336  après ≥ 4 pièces
   24463  après prix <= médiane*1.15
   21666  après prix >= 0.5*moyenne
   21138  après filtre évolution (seuil=-14.4% ou NaN)
   19184  TOTAL retenu (persona)
✅ Exporté : csv_clean/valeurs_foncieres_persona.csv (19184 lignes)


,date_mutation,valeur_fonciere,code_commune,nom_commune,surface_reelle_bati,nombre_pieces_principales,type_local,prix_m2,prix_m2_median_commune,prix_m2_moyen_commune,evolution_2022_2024_%
1,2022-01-03,660000.0,69384,Lyon 4e Arrondissement,120.0,5.0,Appartement,5500.000000,5359.343434,5615.327711,-13.903639
5,2022-01-06,415000.0,69286,Rillieux-la-Pape,103.0,6.0,Maison,4029.126214,3801.408451,3878.150301,-17.852896
7,2022-01-11,444250.0,69071,Curis-au-Mont-d'Or,108.0,4.0,Maison,4113.425926,4643.038949,5288.600114,-7.777643
9,2022-01-06,170000.0,69385,Lyon 5e Arrondissement,68.0,4.0,Appartement,2500.000000,4210.909091,4552.419693,-11.950275
15,2022-01-06,345000.0,69034,Caluire-et-Cuire,77.0,4.0,Appartement,4480.519481,4327.669903,4743.435694,-8.248798
20,2022-01-13,520000.0,69384,Lyon 4e Arrondissement,92.0,4.0,Appartement,5652.173913,5359.343434,5615.327711,-13.903639
27,2022-01-14,435000.0,69034,Caluire-et-Cuire,92.0,4.0,Appartement,4728.260870,4327.669903,4743.435694,-8.248798
29,2022-01-05,314550.0,69034,Caluire-et-Cuire,90.0,4.0,Appartement,3495.000000,4327.669903,4743.435694,-8.248798
30,2022-01-10,543079.0,69381,Lyon 1er Arrondissement,96.0,5.0,Appartement,5657.072917,5482.020030,5644.792815,-14.381573
34,2022-01-06,421500.0,69034,Caluire-et-Cuire,103.0,5.0,Appartement,4092.233010,4327.669903,4743.435694,-8.248798
